Categorical Features

Transaction

*   ProductCD
*   card1
*   card6
*   addr1
*   addr2
*   P_emaildomain
*   R_emaildomain
*   M1 - M9

Categorical Features

Identity

*   DeviceType
*   DeviceInfo
*   id_01 - id_38


In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import datetime
import gc

In [64]:
from sklearn.model_selection import train_test_split
import catboost
"""from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from  sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV"""

'from sklearn.model_selection import cross_val_score, train_test_split\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.linear_model import LogisticRegression\nfrom  sklearn.feature_extraction.text import CountVectorizer\nfrom sklearn.svm import SVC\nfrom sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor\nfrom sklearn.model_selection import GridSearchCV'

Параметры для данного ноутбука:

In [61]:
target = 'isFraud' # Целевой признак
delete_column_list = ['TransactionID','TransactionDT', 'DeviceType','DeviceInfo'] # Список ненужных признаков
is_drop_collinear = True # Удалять ли коррелирующие признаки
coefficient_corr = 0.95 # Коэффициент корреляции, начиная с которого удалять признаки
nan_percent = 0.95 # Степень пустых ячеек, при которой мы относим признак к плохим
is_drop_nan_columns = True # Удалять ли найденные признаки с плохой заполняемостью
bad_fill_category_value = '_NAN_' # Чем заполнять пропуски в категориальных признаках
bad_fill_float_value = 0 # Чем заполнять пропуски в вещественных признаках
bad_fill_numeric_value = 0 # Чем заполнять пропуски в целочисленных признаках
is_need_pca = False #Испольховать ли PCA

#Параметры модели:
CatBoostClassifier_param ={
    #'train_dir':'catboost_clf_find_fraud',
    'eval_metric':'AUC',
    'od_pval':0.05,
    'od_wait':20,
    # early_stopping_rounds:20,
    'random_seed':52,    
    'task_type':'CPU',
    # task_type:'GPU',
    # devices:'0',
    'auto_class_weights':'Balanced',
    'use_best_model':True
}

In [33]:
"""def t(**kw):
  print( kw)
t(**CatBoostClassifier_param)"""

'def t(**kw):\n  print( kw)\nt(**CatBoostClassifier_param)'

Загружаем данные:

In [34]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


"\ndf_train_transaction = pd.read_csv('gdrive/My Drive/data/gp2/train_transaction.csv')\ndf_train_transaction.head()"

In [35]:
def get_data(transaction_csv, identity_csv,is_test = False, delete_column_list=[]):
  df_train_transaction = pd.read_csv(transaction_csv)
  df_train_identity = pd.read_csv(identity_csv)
  df_train = df_train_transaction.merge(df_train_identity, how = 'left',on='TransactionID')
  
  del df_train_identity,df_train_transaction
  gc.collect

  if is_test == False:
    df_y = pd.DataFrame(df_train[target])
    df_train.drop(columns = [target],axis = 1, inplace = True)
  else:
    df_y = pd.DataFrame(df_train['TransactionID'])

  for column in delete_column_list:
    df_train.drop(column, 1,inplace = True)

  return df_train, df_y



Поизучаем данные

In [36]:
df_train, df_y = get_data('gdrive/My Drive/data/gp2/train_transaction.csv', 
                'gdrive/My Drive/data/gp2/train_identity.csv',
                True,[])

Посмотрим на целевую переменную:

In [ ]:
df_train[target].valut_counts()

# Вывод
Выборка несбалансирована

Проверим корреляцию с целевым признаком

In [ ]:
correlations = df_train.corr()[target]
correlations


Посмотрим на признаки, содержащие пропуски:

In [ ]:
def print_count_nan(df):
  row_count = df.shape[0]
  for column in df.columns:
    nan_count = df[column].isna().sum()
    if  nan_count > 0:
      print(column,'  ',nan_count,' (', "{0:.3f}".format(nan_count/row_count), '%)')
      #if row_count == nan_count:
      #  df[column] = 0
    #else:
      #print(column,' нет')

print_count_nan(df_train)

In [37]:
del df_train, df_y 
gc.collect

<function gc.collect>

Загрузим исходный датасет в трейновую выборку

In [38]:
df_train, df_y = get_data('gdrive/My Drive/data/gp2/train_transaction.csv', 
                'gdrive/My Drive/data/gp2/train_identity.csv',
                False,delete_column_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()


Удалим  коррелированные признаки

In [43]:
def remove_collinear_features(x, threshold=0.6):

    
    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)
            
            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
                print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(col.values[0])

    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns = drops)
    
                   
    return x

In [44]:
if is_drop_collinear == True: 
  shape = df_train.shape[1]
  df_trans = remove_collinear_features(df_train, coefficient_corr)
  print('Удалено признаков:')
  print(shape - df_trans.shape[1])

C4 | C1 | 0.97
C4 | C2 | 0.97
C6 | C1 | 0.98
C6 | C2 | 0.97
C6 | C4 | 0.96
C8 | C1 | 0.97
C8 | C2 | 0.98
C8 | C4 | 0.96
C10 | C1 | 0.96
C10 | C2 | 0.97
C10 | C4 | 0.95
C10 | C7 | 0.99
C10 | C8 | 1.0
C11 | C1 | 1.0
C11 | C2 | 0.99
C11 | C4 | 0.97
C11 | C6 | 0.99
C11 | C8 | 0.96
C12 | C7 | 1.0
C12 | C8 | 0.98
C12 | C10 | 0.98
C14 | C1 | 0.95
C14 | C6 | 0.98
C14 | C11 | 0.96
D6 | D4 | 0.96
D7 | D5 | 0.99
D12 | D4 | 1.0
D12 | D6 | 0.98
V21 | V17 | 0.95
V33 | V15 | 0.96
V50 | V31 | 0.98
V50 | V32 | 0.96
V57 | V15 | 0.96
V63 | V42 | 0.95
V71 | V31 | 0.96
V71 | V50 | 0.96
V84 | V21 | 0.96
V84 | V42 | 0.96
V84 | V63 | 0.96
V89 | V28 | 0.97
V92 | V31 | 0.97
V92 | V50 | 0.97
V92 | V71 | 0.96
V94 | V15 | 0.95
V94 | V33 | 0.98
V94 | V51 | 0.95
V101 | V95 | 1.0
V102 | V96 | 1.0
V102 | V97 | 0.99
V103 | V96 | 0.99
V103 | V97 | 1.0
V126 | V95 | 0.98
V126 | V101 | 0.98
V127 | V96 | 0.99
V127 | V97 | 0.98
V127 | V102 | 0.99
V127 | V103 | 0.98
V128 | V96 | 0.98
V128 | V97 | 0.99
V128 | V102 | 0.98
V128 

Разделение столбцов по типам и по заполненностью данными:

In [45]:
print(df_train.dtypes)

TransactionAmt    float64
ProductCD          object
card1               int64
card2             float64
card3             float64
                   ...   
id_34              object
id_35              object
id_36              object
id_37              object
id_38              object
Length: 429, dtype: object


In [46]:
from numpy.core.numerictypes import obj2sctype
def create_column_lists(data, max_nan=0.8):
  column_dict={}
  column_dict['categorical_column'] = []
  column_dict['numeric_column'] = []
  column_dict['float_column'] = []
  column_dict['bad_categorical_column'] = []
  column_dict['bad_numeric_column'] = []
  column_dict['bad_float_column'] = []
  for column in data.columns:
    row_count = data[column].shape[0]
    if data[column].isna().sum()/row_count > max_nan:
      if data[column].dtypes == object:
        column_dict['bad_categorical_column'].append(column)
      else:
        if data[column].dtypes == float:
          column_dict['bad_float_column'].append(column)
        else:
          column_dict['bad_numeric_column'].append(column)
    else:
      if data[column].dtypes == object:
        column_dict['categorical_column'].append(column)
      else:
        if data[column].dtypes == float:
          column_dict['float_column'].append(column)
        else:
          column_dict['numeric_column'].append(column)
  return column_dict




In [47]:
column_dict = create_column_lists(df_train, nan_percent)
for key in column_dict:
  print(key+':')
  print(column_dict[key])

categorical_column:
['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']
numeric_column:
['card1']
float_column:
['TransactionAmt', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64

Разбираемся с почти пустыми столбцами и заполним значениями остальные

In [48]:
def fill_data(data,column_dict,is_drop_nan = False, is_test = False):
  if is_drop_nan == True:
    data.drop(columns=column_dict['bad_float_column']+
            column_dict['bad_numeric_column']+
            column_dict['bad_categorical_column'],axis = 1,inplace = True)
  else:
    for column in column_dict['bad_categorical_column']:
      data[column].fillna(bad_fill_category_value, inplace = True)
    for column in column_dict['bad_float_column']:
      data[column].fillna(bad_fill_float_value, inplace = True)
    for column in column_dict['bad_numeric_column']:
      data[column].fillna(bad_fill_numeric_value, inplace = True)
    if is_test == False:
      column_dict['categorical_column'] = column_dict['categorical_column']+\
        column_dict['bad_categorical_column']
      column_dict['float_column'] = column_dict['float_column']+\
        column_dict['bad_float_column']
      column_dict['numeric_column'] = column_dict['numeric_column']+\
        column_dict['bad_numeric_column']

  """column_dict['bad_categorical_column']=[]
  column_dict['bad_float_column']=[]
  column_dict['bad_numeric_column']=[]"""

  for column in column_dict['categorical_column']:
    data[column].fillna(bad_fill_category_value, inplace = True)
  for column in column_dict['float_column']:
    data[column].fillna(data[column].mean(), inplace = True)
  for column in column_dict['numeric_column']:
    data[column].fillna(data[column].mean(), inplace = True)

  return data, column_dict


In [49]:
df_train,column_dict = fill_data(df_train,column_dict,is_drop_nan_columns, False)

Готовим данные

In [50]:
X_train, X_val, y_train, y_val = train_test_split(df_train, df_y, test_size = 0.3, random_state = 42)


In [51]:
del df_train,df_y
gc.collect

<function gc.collect>

In [52]:
from sklearn.decomposition import PCA
if is_need_pca == True:
  pca = PCA(15)
  pca.fit(X_train[column_dict['float_column']+column_dict['numeric_column']])
  plt.plot(pca.explained_variance_ratio_.cumsum())
  plt.axvline(x=4)
  plt.grid()
  X_train_pca_part = pca.transform(X_train[column_dict['float_column']+column_dict['numeric_column']])
  X_val_pca_part  = pca.transform(X_val[column_dict['float_column']+column_dict['numeric_column']])

CatBoost

In [53]:
!pip install -q catboost shap

In [67]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

#model = CatBoostClassifier()

if is_need_pca == True: 
  CatBoostClassifier_param['eval_set']=X_val_pca_part
  val_pool = catboost.Pool(
    X_val_pca_part,
    label=y_val,
    cat_features=column_dict['categorical_column']#column_dict['float_column']+
  )
  model = CatBoostClassifier(**CatBoostClassifier_param)

  model.fit(X_train_pca_part, y_train) 
  y_pred = model.predict(X_val_pca_part,eval_set=val_pool) 

else:
  #CatBoostClassifier_param['eval_set']=X_val
  train_pool = catboost.Pool(
    X_train,
    label=y_train,
    cat_features=column_dict['categorical_column']#column_dict['numeric_column']+
  )
  val_pool = catboost.Pool(
    X_val,
    label=y_val,
    cat_features=column_dict['categorical_column']#column_dict['float_column']+
  )
  model = CatBoostClassifier(**CatBoostClassifier_param)
  model.fit(#X_train[column_dict['float_column']+column_dict['numeric_column']], 
            #y_train,
            train_pool,
            eval_set=val_pool) 
  y_pred = model.predict(X_val)#[column_dict['float_column']+column_dict['numeric_column']]) 

print(f"{roc_auc_score(y_pred, y_val)}")
 

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.140329
0:	test: 0.8329044	best: 0.8329044 (0)	total: 3.67s	remaining: 1h 1m 12s
1:	test: 0.8473941	best: 0.8473941 (1)	total: 6.67s	remaining: 55m 26s
2:	test: 0.8534126	best: 0.8534126 (2)	total: 10.3s	remaining: 57m 17s
3:	test: 0.8572903	best: 0.8572903 (3)	total: 14.5s	remaining: 1h 21s
4:	test: 0.8620453	best: 0.8620453 (4)	total: 18.1s	remaining: 1h
5:	test: 0.8629267	best: 0.8629267 (5)	total: 21.3s	remaining: 58m 41s
6:	test: 0.8640308	best: 0.8640308 (6)	total: 25.8s	remaining: 1h 53s
7:	test: 0.8660879	best: 0.8660879 (7)	total: 29s	remaining: 59m 51s
8:	test: 0.8683282	best: 0.8683282 (8)	total: 32.3s	remaining: 59m 11s
9:	test: 0.8698130	best: 0.8698130 (9)	total: 35.1s	remaining: 57m 53s
10:	test: 0.8710687	best: 0.8710687 (10)	total: 38.3s	remaining: 57m 26s
11:	test: 0.8722664	best: 0.8722664 (11)	total: 41.3s	remaining: 56m 36s
12:	test: 0.8731872	best: 0.8731872 (12)	total: 44.6s	remaining: 56m 24s
13:	test: 0.8743641	best: 0.8743641 (13)	total: 

In [68]:
if is_need_pca == True: 
  del X_train_pca_part, X_val_pca_part
  gc.collect
else:
  del X_train, X_val,train_pool,val_pool
  gc.collect

In [69]:
print(f"{roc_auc_score(y_pred, y_val)}")

0.6832592914135545


Тестовые данные

In [70]:
df_test, df_y1 = get_data('gdrive/My Drive/data/gp2/test_transaction.csv', 
                'gdrive/My Drive/data/gp2/test_identity.csv',
                True,delete_column_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()


Исправление названий признаков

In [71]:
dict_col = {}
for column in df_test.columns:
  if column.startswith('id-'):
    dict_col[column] = column.replace("-","_")
df_test.rename(columns = dict_col,inplace=True)   

Отказ от всех лишних признаков 

In [72]:
df_test = df_test[column_dict['float_column']+column_dict['numeric_column']+\
                  column_dict['categorical_column']+\
                  column_dict['bad_float_column']+\
                  column_dict['bad_numeric_column']+\
                  column_dict['bad_categorical_column']]

Заполнение пустых ячеек

In [73]:
df_test,column_dict = fill_data(df_test,column_dict,is_drop_nan_columns, True)

Применение PCA

In [74]:
if is_need_pca == True:
  X_test_pca_part = pca.transform(df_test[column_dict['float_column']+column_dict['numeric_column']])


Применение модели

In [75]:
if is_need_pca == True:
  Y_test_pred = model.predict(X_test_pca_part)
else:
  test_pool = catboost.Pool(
    df_test,
    cat_features=column_dict['categorical_column']#column_dict['float_column']+
  )
  Y_test_pred = model.predict(test_pool)#df_test[column_dict['float_column']+column_dict['numeric_column']])


In [76]:
del df_test,test_pool

Формирование датасета результата

In [77]:
submission = pd.DataFrame(columns=['TransactionID'], data=df_y1 )
submission[target] = Y_test_pred
submission.head()
submission.isFraud.value_counts()

0    486466
1     20225
Name: isFraud, dtype: int64

Сохранение файла

In [78]:
submission.to_csv("submission.csv", index = False)

In [ ]:
"""train_pool = catboost.Pool(
    X_train,
    label=y_train,
    cat_features=f
)
val_pool = catboost.Pool(
    X_validate,
    label=y_validate,
    cat_features=f
)
test_pool = catboost.Pool(
    X_test,
    cat_features=f
)"""